<a href="https://colab.research.google.com/github/Zhangmingyang-Su/Click-Through-Rate-Prediction/blob/master/deepctr_DeepFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepFM Model for CTR Prediction

##1. Load File

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving filtered_train.csv to filtered_train.csv
User uploaded file "filtered_train.csv" with length 67056673 bytes


##1.1 Install Deepctr 

In [ ]:
!pip install deepctr[gpu]

     |████████████████████████████████| 92kB 2.5MB/s 
     |████████████████████████████████| 516.2MB 17kB/s 


##1.2 Data Transformation

In [ ]:
# import package from deepctr
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names

In [ ]:
# read csv file
dataset = pd.read_csv("filtered_train.csv")
dataset.head()

,Unnamed: 0,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,128037,1.107451e+19,0,14102101,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,10dcdfb1,88fe1d5d,1,0,20352,320,50,2333,0,39,-1,157
1,491755,1.240012e+19,0,14102103,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,9f351a94,3db9fde9,1,0,15701,320,50,1722,0,35,100084,79
2,470924,1.059801e+19,0,14102103,1005,1,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,158aae2e,c144e605,1,0,17614,320,50,1993,2,1063,-1,33
3,791624,1.844219e+18,0,14102104,1005,1,856e6d3f,58a89a43,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,87f758ac,0eb711ec,1,0,16208,320,50,1800,3,167,100075,23
4,491263,1.236016e+19,1,14102103,1005,1,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,d55aa41c,3bd9e8e7,1,0,17614,320,50,1993,2,1063,-1,33


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        404290 non-null  int64  
 1   id                404290 non-null  float64
 2   click             404290 non-null  int64  
 3   hour              404290 non-null  float64
 4   C1                404290 non-null  float64
 5   banner_pos        404290 non-null  float64
 6   site_id           404290 non-null  int64  
 7   site_domain       404290 non-null  int64  
 8   site_category     404290 non-null  int64  
 9   app_id            404290 non-null  int64  
 10  app_domain        404290 non-null  int64  
 11  app_category      404290 non-null  int64  
 12  device_id         404290 non-null  object 
 13  device_ip         404290 non-null  object 
 14  device_model      404290 non-null  int64  
 15  device_type       404290 non-null  float64
 16  device_conn_type  40

In [ ]:
# features for embedding 
Sparse_feature = ["site_id", "site_domain", "site_category", "app_id", "app_domain", "app_category", "device_model"]

In [ ]:
# features for embedding 
Dense_feature = ["hour", "C1", "banner_pos", "device_type", "device_conn_type", "C14", "C15", "C16", "C17", "C18", "C19", "C21"]

In [ ]:
target = ["click"]

In [ ]:
# label encoding for different char categorical features
def Label(x):
  for feat in x:
    lbe = LabelEncoder()
    dataset[feat] = lbe.fit_transform(dataset[feat])
  return dataset

dataset = Label(Sparse_feature)
dataset.head()

,Unnamed: 0,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,128037,1.107451e+19,0,0.001083,0.363636,0.142857,1939,1096,20,2060,63,0,a99f214a,10dcdfb1,2331,0.2,0.0,0.844051,0.221239,0.02988,0.839698,0.000000,0.003322,-1,0.614173
1,491755,1.240012e+19,0,0.003250,0.363636,0.000000,260,2080,1,2060,63,0,a99f214a,9f351a94,1064,0.2,0.0,0.647541,0.221239,0.02988,0.608696,0.000000,0.001107,100084,0.307087
2,470924,1.059801e+19,0,0.003250,0.363636,0.142857,1879,1322,20,2060,63,0,a99f214a,158aae2e,3271,0.2,0.0,0.728367,0.221239,0.02988,0.711153,0.666667,0.570321,-1,0.125984
3,791624,1.844219e+18,0,0.004334,0.363636,0.142857,1145,769,20,2060,63,0,a99f214a,87f758ac,260,0.2,0.0,0.668962,0.221239,0.02988,0.638185,1.000000,0.074197,100075,0.086614
4,491263,1.236016e+19,1,0.003250,0.363636,0.142857,1879,1322,20,2060,63,0,a99f214a,d55aa41c,1035,0.2,0.0,0.728367,0.221239,0.02988,0.711153,0.666667,0.570321,-1,0.125984


In [ ]:
# Scaling dense features to range(0, 1)
def Scale(x):
  mms = MinMaxScaler(feature_range=(0, 1))
  dataset[x] = mms.fit_transform(dataset[x])
  return dataset
dataset = Scale(Dense_feature)
dataset.head()

,Unnamed: 0,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,128037,1.107451e+19,0,0.001083,0.363636,0.142857,1939,1096,20,2060,63,0,a99f214a,10dcdfb1,2331,0.2,0.0,0.844051,0.221239,0.02988,0.839698,0.000000,0.003322,-1,0.614173
1,491755,1.240012e+19,0,0.003250,0.363636,0.000000,260,2080,1,2060,63,0,a99f214a,9f351a94,1064,0.2,0.0,0.647541,0.221239,0.02988,0.608696,0.000000,0.001107,100084,0.307087
2,470924,1.059801e+19,0,0.003250,0.363636,0.142857,1879,1322,20,2060,63,0,a99f214a,158aae2e,3271,0.2,0.0,0.728367,0.221239,0.02988,0.711153,0.666667,0.570321,-1,0.125984
3,791624,1.844219e+18,0,0.004334,0.363636,0.142857,1145,769,20,2060,63,0,a99f214a,87f758ac,260,0.2,0.0,0.668962,0.221239,0.02988,0.638185,1.000000,0.074197,100075,0.086614
4,491263,1.236016e+19,1,0.003250,0.363636,0.142857,1879,1322,20,2060,63,0,a99f214a,d55aa41c,1035,0.2,0.0,0.728367,0.221239,0.02988,0.711153,0.666667,0.570321,-1,0.125984


##2. Model Apply and Evaluation

In [ ]:
# embedding all features above
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=dataset[feat].nunique(),embedding_dim=4)
                           for i,feat in enumerate(Sparse_feature)] + [DenseFeat(feat, 1,)
                          for feat in Dense_feature]

In [ ]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
feature_names

['site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_model',
 'hour',
 'C1',
 'banner_pos',
 'device_type',
 'device_conn_type',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C21']

In [ ]:
# train and test split
train, test = train_test_split(dataset, test_size=0.2)
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}


In [ ]:
# Apply DeepFM Model
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",
                metrics=['binary_crossentropy'], )

In [ ]:
# Model Evaluation
history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=20, verbose=2, validation_split=0.2,)
pred_ans = model.predict(test_model_input, batch_size=256)

Epoch 1/20


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1011/1011 - 6s - loss: 0.4120 - binary_crossentropy: 0.4117 - val_loss: 0.4066 - val_binary_crossentropy: 0.4058
Epoch 2/20
1011/1011 - 6s - loss: 0.4014 - binary_crossentropy: 0.4003 - val_loss: 0.4055 - val_binary_crossentropy: 0.4041
Epoch 3/20
1011/1011 - 6s - loss: 0.3983 - binary_crossentropy: 0.3966 - val_loss: 0.4060 - val_binary_crossentropy: 0.4041
Epoch 4/20
1011/1011 - 6s - loss: 0.3961 - binary_crossentropy: 0.3939 - val_loss: 0.4072 - val_binary_crossentropy: 0.4049
Epoch 5/20
1011/1011 - 6s - loss: 0.3940 - binary_crossentropy: 0.3915 - val_loss: 0.4082 - val_binary_crossentropy: 0.4055
Epoch 6/20
1011/1011 - 6s - loss: 0.3922 - binary_crossentropy: 0.3894 - val_loss: 0.4097 - val_binary_crossentropy: 0.4068
Epoch 7/20
1011/1011 - 6s - loss: 0.3902 - binary_crossentropy: 0.3872 - val_loss: 0.4115 - val_binary_crossentropy: 0.4084
Epoch 8/20
1011/1011 - 6s - loss: 0.3885 - binary_crossentropy: 0.3854 - val_loss: 0.4130 - val_binary_crossentropy: 0.4098
Epoch 9/20
1011/101

In [ ]:
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

test AUC 0.7216
